In [1]:
# 로지스틱 회귀분석

In [5]:
# titanic 데이터를 분할하고, train 데이터로 로지스틱 회귀분석을 만들어보자

In [6]:
import pandas as pd
import numpy as np

In [23]:
df_t = pd.read_csv('data/titanic.csv')

In [24]:
# 로지스틱 회귀분석은 결측치 제거 필요
df_t.info()
df_t['age'] = df_t['age'].fillna(df_t['age'].median())
df_t['embark_town'] = df_t['embark_town'].fillna(df_t['embark_town'].value_counts().index[0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   adult_male   891 non-null    bool   
 10  embark_town  889 non-null    object 
dtypes: bool(1), float64(2), int64(4), object(4)
memory usage: 70.6+ KB


In [25]:
# 중복된 변수 제거
df_t = df_t.drop(['pclass', 'embarked'], axis = 1)
# 더미변수화
df_t = pd.get_dummies(df_t, columns=['sex', 'class', 'embark_town'])

In [26]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   survived                 891 non-null    int64  
 1   age                      891 non-null    float64
 2   sibsp                    891 non-null    int64  
 3   parch                    891 non-null    int64  
 4   fare                     891 non-null    float64
 5   adult_male               891 non-null    bool   
 6   sex_female               891 non-null    uint8  
 7   sex_male                 891 non-null    uint8  
 8   class_First              891 non-null    uint8  
 9   class_Second             891 non-null    uint8  
 10  class_Third              891 non-null    uint8  
 11  embark_town_Cherbourg    891 non-null    uint8  
 12  embark_town_Queenstown   891 non-null    uint8  
 13  embark_town_Southampton  891 non-null    uint8  
dtypes: bool(1), float64(2), in

In [28]:
df_t['adult_male'].value_counts()
df_t = df_t.astype(float)

In [29]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   survived                 891 non-null    float64
 1   age                      891 non-null    float64
 2   sibsp                    891 non-null    float64
 3   parch                    891 non-null    float64
 4   fare                     891 non-null    float64
 5   adult_male               891 non-null    float64
 6   sex_female               891 non-null    float64
 7   sex_male                 891 non-null    float64
 8   class_First              891 non-null    float64
 9   class_Second             891 non-null    float64
 10  class_Third              891 non-null    float64
 11  embark_town_Cherbourg    891 non-null    float64
 12  embark_town_Queenstown   891 non-null    float64
 13  embark_town_Southampton  891 non-null    float64
dtypes: float64(14)
memory usag

In [30]:
df_t['adult_male'].value_counts() # True 1.0 False 0.0

1.0    537
0.0    354
Name: adult_male, dtype: int64

In [31]:
# 설명변수(X), 타겟변수(y) 분리
X = df_t.drop(['survived'], axis = 1)
y = df_t['survived']

In [33]:
# 홀드아웃 기법
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = 2020)
# test_size = 0.2 기준으로 1차 분류
# 분류된 내부를 stratify 기준으로 2차 분류

In [46]:
y.value_counts()
# stratify = y
# 0.0 
print(549/(549+342))
# 1.0 
print(342/(549+342))

print(len(X_train))
print(len(X_test))
print(712/(712+179))
print(179/(712+179))
print(len(y_train))
print(len(y_test))

0.6161616161616161
0.3838383838383838
712
179
0.7991021324354658
0.20089786756453423
712
179


In [48]:
import statsmodels.api as sm
#help(sm.Logit)
model = sm.Logit(y_train, X_train) # 순서 중요
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.409459
         Iterations 7


In [49]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:               survived   No. Observations:                  712
Model:                          Logit   Df Residuals:                      701
Method:                           MLE   Df Model:                           10
Date:                Tue, 20 Sep 2022   Pseudo R-squ.:                  0.3849
Time:                        22:59:33   Log-Likelihood:                -291.53
converged:                       True   LL-Null:                       -473.99
Covariance Type:            nonrobust   LLR p-value:                 2.720e-72
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
age                        -0.0184      0.009     -1.960      0.050      -0.037   -3.62e-06
sibsp                      -0.5172      0.133     -3.875      0.000      -0.779      -0.256


In [50]:
# 성능척도_1 AIC
print("model AIC : ", "{:.5f}".format(result.aic))

model AIC :  605.06963
